In [2]:
# To do:

# Define is the team still in it?
# - define the behaviour when in risk, and whether to double down, how long to hold, when to cut loss


In [3]:
import pandas as pd
import time
import logging
from pythonjsonlogger import jsonlogger
from flumine import FlumineSimulation, clients
from collections import OrderedDict, deque, defaultdict
from flumine import BaseStrategy
from flumine.order.trade import Trade
from flumine.order.order import OrderStatus
from flumine.order.ordertype import LimitOrder
from flumine.utils import get_price
from logging_setup import build_logger
import random

import time
from datetime import timedelta

class HugoStrat(BaseStrategy):
    """
    Example strateg
    """
    def __init__(self, enter_threshold, exit_threshold, order_hold, price_add, log_root, log_level, *a, **k):
        self.log = build_logger(log_root,log_level)  # logs/trades.log, rotated nightly
        super().__init__(*a, **k)
        self.hist = defaultdict(lambda: deque(maxlen=400))  # per runner
        self.enter_threshold = enter_threshold
        self.exit_threshold = exit_threshold
        self.order_hold = order_hold
        self.price_add = price_add
        # self.dev = dev
        # self.lookback = lookback
        # self.take_prof_price_diff = take_prof_price_diff
        self.startdt = None
        self._last = {}    # order_id -> last size_matched
        self.rows = []     # collected fills: [market_id, selection_id, time, size, price, side, order_id]
        self.pnl = 0.0     


    def add_market(self, market):
        self.log.info("ADD", market.market_id, market.event_name, market.event_type_id)
    
    def check_market_book(self, market, market_book):
        if market_book.status == "OPEN" and market_book.inplay:
            return True
    
    def _price_now(self, r):
        return r.last_price_traded
    
    def _price_n_secs_ago(self, key, now_dt, n=5):
        cutoff = now_dt - timedelta(seconds=n)
        dq = self.hist.get(key)
        if not dq: return None
        # find the latest sample at/before cutoff
        for t,p in reversed(dq):
            if t <= cutoff:
                return p
        return None

    def avg_back_odds(self, market, selection_id):
        total_stake, weighted = 0, 0
        for order in market.blotter:
            if order.selection_id == selection_id and order.side == "BACK":
                matched = order.size_matched
                if matched > 0:
                    total_stake += matched
                    weighted += matched * order.average_price_matched
        return weighted / total_stake if total_stake else None
    
    def process_market_book(self, market, market_book):
        if not self.startdt: self.startdt = market_book.publish_time
        elapsed = market_book.publish_time.timestamp() - self.startdt.timestamp()
        self.log.debug(f"process_market_book: {market.event_name}, time elapse {elapsed},  publishtime:{market_book.publish_time}")

# wait until 1 hour done then hedge out
        if elapsed > 1:
            self.log.debug(f"elapsed time {elapsed} > 3200, place bets return")
            for r in market_book.runners:
                now_dt = market_book.publish_time  # simulation “now”
                key = (market.market_id, r.selection_id)
                p = self._price_now(r)
                if p and p < self.enter_threshold:
                    runner_context = self.get_runner_context(market.market_id, r.selection_id, r.handicap)
                    if runner_context.live_trade_count == 0:
                        self.log.info(f"price less than thresh, no live trades, placing order: {r.selection_id}")
                        # back at current best lay price
                        back = round(get_price(r.ex.available_to_lay, 0) + self.price_add,2)
                        # create trade
                        trade = Trade(market_book.market_id, r.selection_id, r.handicap,self, notes={"entry_px": back})
                        order = trade.create_order(side="BACK", order_type=LimitOrder(back, self.context["stake"]))
                        market.place_order(order)
                        self.log.info({"PLACE ORDER":market.market_id,"price":back,"event_name":market.event_name})

                if not p : return
                if p > self.exit_threshold:
                    self.hedge_selection(r, market)

    def avg_back_odds(self, market, selection_id):
        total_stake, weighted = 0, 0
        for order in market.blotter:
            if order.selection_id == selection_id and order.side == "BACK":
                matched = order.size_matched
                if matched > 0:
                    total_stake += matched
                    weighted += matched * order.average_price_matched
        return weighted / total_stake if total_stake else None
    
    def hedge_selection(self,r, market):        
        backs = [o for o in market.blotter if o.selection_id==r.selection_id and o.side=="BACK" and o.size_matched>0]
        stake = sum(o.size_matched for o in backs)
        best_lay = get_price(r.ex.available_to_lay, 0)
        if not best_lay: return
        av = self.avg_back_odds(market, r.selection_id)
        if not av: return
        # hedge size so Pwin == Plose
        hsize = (av*stake - stake) / best_lay
        if hsize <= 0: return
        self.log.info(f"Closing risk : runner {r.selection_id}, {hsize} @ {best_lay}")
        trade = Trade(market.market_id, r.selection_id, r.handicap, self)
        order = trade.create_order("LAY", LimitOrder(best_lay, round(hsize,2), persistence_type="LAPSE"))
        market.place_order(order)
 
    
    def process_orders(self, market, orders):
        # kill order if unmatched in market for greater than 2 seconds
        for order in orders:
            if order.status == OrderStatus.EXECUTABLE:
                if order.elapsed_seconds and order.elapsed_seconds > self.order_hold:
                     market.cancel_order(order)

    def process_order(self, order):
        # fully matched
        if order.status == OrderStatus.EXECUTION_COMPLETE:
            a = {"Order fully matched": order.selection_id,"avg_px":order.average_price_matched,
                 "size":order.size_matched}
            self.log.info(a)

        # partially matched
        elif order.size_matched and order.size_remaining:
            a = {"Partial match":order.selection_id,
                 "matched":order.size_matched,
                 "remaining":order.size_remaining,
                 "avg_px":order.average_price_matched}
            self.log.info(a)

    def process_closed_market(self, market, market_book):
        self.pnl = 0.0
        self.log.info(f"Processing closed market: {market.event_name}, {market.market_id}")
        for order in market.blotter:
            self.pnl += order.profit
            self.log.info(f"Order PNL {order.profit}, av size matched: {order.size_matched} av price matched: {order.average_price_matched}, date_time_created: {order.date_time_created}")
            
        self.log.warning(f"Total pnl for market:{market.event_name}, {market.market_id}, : PNL :: {self.pnl}")
            
    

In [4]:
markets = pd.read_csv("ADVeventid_marketid_AUG24.csv")["path"].tolist()

In [5]:
def run_once(market, max_order_exposure, max_selection_exposure, stake, enter_threshold, exit_threshold, order_hold, price_add):
    client = clients.SimulatedClient()
    framework = FlumineSimulation(client=client)

    lists =[]
    lists.append(market)
    
    strategy = HugoStrat(   
        market_filter={"markets": lists},
        max_order_exposure=max_order_exposure,
        max_selection_exposure=max_selection_exposure,
        context={"stake": stake},
        enter_threshold=enter_threshold,
        exit_threshold=exit_threshold,
        order_hold=order_hold,
        price_add=price_add,
        log_root="./logs/backtest/",
        log_level="E")

    framework.add_strategy(strategy)
    framework.run()
    pnl = getattr(strategy, "pnl")
    del framework, client, strategy
    return pnl

In [51]:
def run_one_trial_drawdown_optim(max_order_exposure, max_selection_exposure, stake, enter_threshold, exit_threshold, order_hold, price_add):    
    pnls = []
    for i in markets:
        pnl = run_once(i,max_order_exposure=max_order_exposure,max_selection_exposure=max_selection_exposure, stake=stake,
                       enter_threshold=enter_threshold, exit_threshold=exit_threshold,  order_hold=order_hold, price_add=price_add)
        pnls.append(pnl)
        print(i, pnl)
    return pnls

def sum_losses(equity):
    result = sum(abs(x) for x in equity if x <= 0)
    return result


pnls = run_one_trial_drawdown_optim(30,90,2,1.2,5.9,17,0.01)
print(sum(pnls), sum_losses(pnls))

 # arams={'max_order_exposure': 30, 'max_selection_exposure': 90, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 17, 'price_add': 0.01}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 2}, intermediate_values={}, distributions={'max_order_exposure': IntDistribution(high=40, log=False, low=5, step=5), 'max_selection_exposure': IntDistribution(high=200, log=False, low=20, step=10), 'stake': FloatDistribution(high=15.0, log=False, low=1.0, step=1.0), 'enter_threshold': FloatDistribution(high=4.0, log=False, low=1.02, step=0.01), 'exit_threshold': FloatDistribution(high=8.0, log=False, low=2.0, step=0.1), 'order_hold': IntDistribution(high=37, log=False, low=1, step=4), 'price_add': FloatDistribution(high=0.08, log=False, low=-0.01, step=0.01)}, trial_id=75, value=None)

hist_data/ADVANCED/2025/Feb/12/33524813/1.232245129 5.56
hist_data/ADVANCED/2025/Feb/12/33475068/1.231541189 0.65
hist_data/ADVANCED/2024/Aug/20/33508295/1.232019003 3.3600000000000008
hist_data/ADVANCED/2024/Aug/20/33504642/1.231954918 0.32
hist_data/ADVANCED/2024/Aug/20/33508296/1.232018919 3.49
hist_data/ADVANCED/2024/Aug/20/33508297/1.232019056 11.200000000000003
hist_data/ADVANCED/2024/Aug/18/33501074/1.231895741 3.9800000000000013
hist_data/ADVANCED/2024/Aug/18/33501073/1.231895769 9.21
hist_data/ADVANCED/2024/Aug/18/33504088/1.231946357 1.6400000000000001
hist_data/ADVANCED/2024/Aug/18/33500547/1.231887491 -23.13999999999999
hist_data/ADVANCED/2024/Aug/18/33503359/1.231929101 4.300000000000001
hist_data/ADVANCED/2024/Aug/27/33528205/1.232298775 0.48
hist_data/ADVANCED/2024/Aug/27/33525729/1.232260815 3.3300000000000005
hist_data/ADVANCED/2024/Aug/27/33525672/1.232259812 3.66
hist_data/ADVANCED/2024/Aug/27/33527290/1.232283631 1.04
hist_data/ADVANCED/2024/Aug/27/33527093/1.232301

In [7]:
import optuna
from optuna.samplers import NSGAIISampler
from optuna.pruners import SuccessiveHalvingPruner


def objective(trial):
    max_order_exposure = trial.suggest_int("max_order_exposure", 5, 40, step=5)
    max_selection_exposure = trial.suggest_int("max_selection_exposure", 20, 200, step=10)
    stake = trial.suggest_float("stake", 1.0, 15.0, step=1)
    enter_threshold = trial.suggest_float("enter_threshold", 1.02, 4, step=0.01)
    exit_threshold = trial.suggest_float("exit_threshold", 2, 8, step=0.1)
    order_hold = trial.suggest_int("order_hold", 1, 37, step=4)
    price_add = trial.suggest_float("price_add", -0.01, 0.08 , step=0.01)

    equity = run_one_trial_drawdown_optim(
        max_order_exposure=max_order_exposure,
        max_selection_exposure=max_selection_exposure,
        stake=stake,
        enter_threshold=enter_threshold,
        exit_threshold=exit_threshold,
        order_hold=order_hold,
        price_add=price_add
    )
    
    pnl_sum = sum(equity)

    sum_loss = sum_losses(equity)
    print("pnls: ", equity, "\nsum:", pnl_sum, "sum losses : -", sum_loss)
    return pnl_sum, sum_loss

study = optuna.create_study(directions=["maximize", "minimize"],sampler=NSGAIISampler(population_size=28))
study.optimize(objective, n_trials=300, n_jobs=1)

# Choose a Pareto-optimal trial (e.g., lowest MDD above a PnL floor)
best = min(study.best_trials, key=lambda t: (t.values[1], -t.values[0]))
print("Best (PnL, MDD):", best.values)
print("Params:", best.params)


[I 2025-09-13 21:29:07,622] A new study created in memory with name: no-name-e05489ce-ad4a-41e9-969f-f38399a5a36a


hist_data/ADVANCED/2024/Aug/11/33477738/1.231582170 0.0
hist_data/ADVANCED/2024/Aug/21/33511213/1.232049325 0.0
hist_data/ADVANCED/2024/Aug/28/33532598/1.232352479 0.0
hist_data/ADVANCED/2024/Aug/27/33528205/1.232298775 0.0
hist_data/ADVANCED/2024/Aug/22/33514178/1.232089391 0.0
hist_data/ADVANCED/2024/Aug/11/33480307/1.231615888 0.0
hist_data/ADVANCED/2024/Aug/9/33474911/1.231539716 0.0
hist_data/ADVANCED/2024/Aug/6/33467052/1.231432440 0.0
hist_data/ADVANCED/2024/Aug/12/33483444/1.231661434 0.0
hist_data/ADVANCED/2024/Aug/16/33495564/1.231819462 0.0
hist_data/ADVANCED/2024/Aug/22/33514172/1.232089444 0.0
hist_data/ADVANCED/2024/Aug/2/33455130/1.231243232 0.0
hist_data/ADVANCED/2024/Aug/4/33462715/1.231345275 0.0
hist_data/ADVANCED/2024/Aug/4/33462952/1.231371294 0.0
hist_data/ADVANCED/2024/Aug/14/33486371/1.231705583 0.0
hist_data/ADVANCED/2024/Aug/26/33508011/1.232012797 0.0
hist_data/ADVANCED/2024/Aug/30/33527208/1.232283868 0.0
hist_data/ADVANCED/2024/Aug/5/33466781/1.231426333 0.

[I 2025-09-13 21:31:11,589] Trial 0 finished with values: [0.0, 0.0] and parameters: {'max_order_exposure': 5, 'max_selection_exposure': 40, 'stake': 11.0, 'enter_threshold': 1.84, 'exit_threshold': 6.800000000000001, 'order_hold': 33, 'price_add': 0.03}.


hist_data/ADVANCED/2024/Aug/17/33495342/1.231818131 0.0
pnls:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
sum: 0.0 sum losses : - 0.0
hist_data/ADVANCED/2024/Aug/4/33463142/1.231355726 12.49
hist_data/ADVANCED/2024/Aug/21/33508163/1.232013794 3.640000000000002
hist_data/ADVANCED/2024/Aug/11/33477757/1.231582357 9.709999999999994
hist_data/ADVANCED/2024/Aug/4/33464867/1.231395388 -8.61
hist_data/ADVANCED/2024/Aug/29/33534236/1.232375043 31.660000000000004
hist_data/ADVANCED/2024/Aug/2/33457992/1.231282946 11.510000000000007
hist_data/ADVANCED/2024/Aug/14/33483335/1.231659753 0.0
hist_data/ADVANCED/2024/Aug/4/33462723/1.231345620 -0.9600000000000009
hist_data/ADVANCED/2024/Aug/2/33457835/1.231282404 0.0
hist_data/ADVANCED/2024/Aug/4/33462743/1.231344137 22.589999999999996
hist_data/ADVANCED/2024/Aug/4/33462

[I 2025-09-13 21:34:08,296] Trial 1 finished with values: [89.74, 227.97000000000003] and parameters: {'max_order_exposure': 5, 'max_selection_exposure': 70, 'stake': 2.0, 'enter_threshold': 3.95, 'exit_threshold': 3.4000000000000004, 'order_hold': 17, 'price_add': 0.0}.


...

hist_data/ADVANCED/2024/Aug/18/33501073/1.231895769 0.0
pnls:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
sum: 0.0 sum losses : - 0.0
hist_data/ADVANCED/2024/Aug/31/33540261/1.232462894 0.0
hist_data/ADVANCED/2024/Aug/14/33485740/1.231691586 63.66999999999997
hist_data/ADVANCED/2024/Aug/10/33482920/1.231653447 0.0
hist_data/ADVANCED/2024/Aug/28/33527284/1.232283832 1.32
hist_data/ADVANCED/2024/Aug/6/33467903/1.231442166 31.939999999999998
hist_data/ADVANCED/2024/Aug/28/33532565/1.232352477 -24.150000000000006
hist_data/ADVANCED/2024/Aug/6/33469869/1.231469608 10.940000000000001
hist_data/ADVANCED/2024/Aug/26/33508012/1.232012579 -36.96000000000001
hist_data/ADVANCED/2024/Aug/4/33462734/1.231344548 31.49
hist_data/ADVANCED/2024/Aug/2/33458074/1.231283997 39.53
hist_data/ADVANCED/2024/Aug/24/33517182/1.2321

[I 2025-09-14 09:24:18,518] Trial 152 finished with values: [116.70000000000003, 573.49] and parameters: {'max_order_exposure': 15, 'max_selection_exposure': 180, 'stake': 11.0, 'enter_threshold': 3.3200000000000003, 'exit_threshold': 7.7, 'order_hold': 9, 'price_add': 0.060000000000000005}.


hist_data/ADVANCED/2024/Aug/8/33472583/1.231501640 -18.870000000000008
pnls:  [0.0, 63.66999999999997, 0.0, 1.32, 31.939999999999998, -24.150000000000006, 10.940000000000001, -36.96000000000001, 31.49, 39.53, -3.48, 14.740000000000007, 92.32000000000004, 0.01, -66.26, 25.13999999999998, 6.580000000000001, -0.19999999999999885, 26.75, 0.0, 0.07, -56.53999999999999, -14.429999999999987, -8.590000000000002, 8.919999999999998, 17.889999999999997, 102.6, -46.73, 49.58999999999999, -11.789999999999994, 0.0, -43.95, 89.59, -44.19, 0.1700000000000012, 14.08, 60.540000000000006, -47.24, -11.590000000000002, -51.400000000000006, -66.86999999999995, -20.25, 1.769999999999999, 0.54, -18.870000000000008] 
sum: 116.70000000000003 sum losses : - 573.49
hist_data/ADVANCED/2024/Aug/14/33483335/1.231659753 0.0
hist_data/ADVANCED/2024/Aug/8/33472583/1.231501640 0.42000000000000004
hist_data/ADVANCED/2024/Aug/24/33517182/1.232130684 0.0
hist_data/ADVANCED/2024/Aug/1/33454955/1.231241311 0.0
hist_data/ADVA

[W 2025-09-14 09:24:54,578] Trial 153 failed with parameters: {'max_order_exposure': 5, 'max_selection_exposure': 20, 'stake': 2.0, 'enter_threshold': 1.05, 'exit_threshold': 3.1, 'order_hold': 9, 'price_add': 0.060000000000000005} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/anaconda3/envs/betenv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/ry/2xyb1vq17fn9t5zdq95yj2g80000gn/T/ipykernel_78339/1006145758.py", line 15, in objective
    equity = run_one_trial_drawdown_optim(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ry/2xyb1vq17fn9t5zdq95yj2g80000gn/T/ipykernel_78339/2752813999.py", line 4, in run_one_trial_drawdown_optim
    pnl = run_once(i,max_order_exposure=max_order_exposure,max_selection_exposure=max_selection_exposure, stake=stake,
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
# Trial 82 f [59.2, 83.27000000000001] and parameters: {'max_order_exposure': 25, 'max_selection_exposure': 20, 'stake': 7.0, 'enter_threshold': 3.14, 'exit_threshold': 4.0, 'order_hold': 9, 'price_add': 0.0}.
# Trial 81 f [81.99000000000001, 14.479999999999997] and parameters: {'max_order_exposure': 15, 'max_selection_exposure': 30, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 3.1, 'order_hold': 13, 'price_add': 0.01}.
# Trial 75 f [87.41000000000003, 7.68] and parameters: {'max_order_exposure': 30, 'max_selection_exposure': 90, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 17, 'price_add': 0.01}.
# Trial 65 f [53.540000000000006, 21.71] and parameters: {'max_order_exposure': 30, 'max_selection_exposure': 30, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 13, 'price_add': 0.01}.
# Trial 56 f [57.92, 73.95] and parameters: {'max_order_exposure': 15, 'max_selection_exposure': 20, 'stake': 1.0, 'enter_threshold': 1.5, 'exit_threshold': 3.8, 'order_hold': 5, 'price_add': 0.01}.
# Trial 53 f [92.24, 54.10000000000001] and parameters: {'max_order_exposure': 25, 'max_selection_exposure': 20, 'stake': 7.0, 'enter_threshold': 3.48, 'exit_threshold': 8.0, 'order_hold': 9, 'price_add': -0.01}.
# Trial 43 f [682.89, 516.9399999999999] and parameters: {'max_order_exposure': 10, 'max_selection_exposure': 160, 'stake': 6.0, 'enter_threshold': 3.95, 'exit_threshold': 5.0, 'order_hold': 17, 'price_add': 0.049999999999999996}.


In [44]:
# Choose a Pareto-optimal trial (e.g., lowest MDD above a PnL floor)
best = min(study.best_trials, key=lambda t: (t.values[1], -t.values[0]))
print("Best (PnL, MDD):", best.values)
print("Params:", best.params)


Best (PnL, MDD): [0.0, 0.0]
Params: {'max_order_exposure': 5, 'max_selection_exposure': 40, 'stake': 11.0, 'enter_threshold': 1.84, 'exit_threshold': 6.800000000000001, 'order_hold': 33, 'price_add': 0.03}


In [46]:
lister = [ i for i in study.best_trials if i.values[0] != 0.0]
for i in lister:
    print("\n", i)


 FrozenTrial(number=10, state=1, values=[142.37, 127.95], datetime_start=datetime.datetime(2025, 9, 13, 21, 58, 33, 167162), datetime_complete=datetime.datetime(2025, 9, 13, 22, 0, 6, 921394), params={'max_order_exposure': 35, 'max_selection_exposure': 30, 'stake': 15.0, 'enter_threshold': 1.45, 'exit_threshold': 7.7, 'order_hold': 1, 'price_add': -0.01}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 0}, intermediate_values={}, distributions={'max_order_exposure': IntDistribution(high=40, log=False, low=5, step=5), 'max_selection_exposure': IntDistribution(high=200, log=False, low=20, step=10), 'stake': FloatDistribution(high=15.0, log=False, low=1.0, step=1.0), 'enter_threshold': FloatDistribution(high=4.0, log=False, low=1.02, step=0.01), 'exit_threshold': FloatDistribution(high=8.0, log=False, low=2.0, step=0.1), 'order_hold': IntDistribution(high=37, log=False, low=1, step=4), 'price_add': FloatDistribution(high=0.08, log=False, low=-0.01, step=0.01)}, trial_id=10, valu